# Setup

In [1]:
# Check the Python version to ensure compatibility
import sys
print(sys.version)

3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]


In [2]:
# Install the 'swarm' library from GitHub and the 'requests' library for HTTP requests
!pip install git+https://github.com/openai/swarm.git
!pip install requests

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-ewy8ack7
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-ewy8ack7
  Resolved https://github.com/openai/swarm.git to commit 0c82d7d868bb8e2d380dfd2a319b5c3a1f4c0cb9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
# Import the 'userdata' module to securely handle user data (API keys)
from google.colab import userdata

# Retrieve the OpenAI API key and News API key from user data
openai_api_key = userdata.get('genai_course')
news_api_key = userdata.get('news_api')

In [4]:
# Import required libraries
import os
from swarm import Swarm, Agent
import requests
from IPython.display import display, Markdown

In [5]:
# Set the API keys as environment variables
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['NEWS_API_KEY'] = news_api_key

# Agent 1 - Greeting Agent

In [6]:
# Define the Greeting Agent
greeting_agent = Agent(
    name="Greeting Agent",
    instructions="You are a friendly assistant that greets the user and tells a joke.",
)

# Agent 2 - News Agent

In [7]:
def fetch_news(country="us", category=None):
    """
    Fetches the top news headlines for a given country and category.

    Args:
        country (str): The country code (default is "us").
        category (str): The news category (optional).

    Returns:
        str: A string containing the top news headlines or an error message.
    """
    base_url = "https://newsapi.org/v2/top-headlines"
    params = {
        "country": country,
        "apiKey": news_api_key
    }

    # Add a category if provided
    if category:
        params["category"] = category

    # Make a GET request to the News API
    response = requests.get(base_url, params=params)
    data = response.json()

    # Check if the request was successful
    if response.status_code == 200 and 'articles' in data:
        # Extract the titles of the first 10 articles
        headlines = [article['title'] for article in data['articles'][:10]]
        return f"Here are the top news for {country.upper()} ({category or 'all categories'}):\n" + "\n".join(headlines)
    else:
        # Return an error message if the request failed
        return f"Sorry, I couldn't fetch the news at the moment: {data.get('message', 'Unknown error')}"

In [8]:
# Define the News Agent
news_agent = Agent(
    name="News Agent",
    instructions="You provide the top news headlines for a given country and category.",
    functions=[fetch_news]
)

# Orchestrating the agents

In [9]:
# Define function that transfers the task to the agent
def transfer_to_agent(agent_name):
  agents = {
      "Greeting Agent": greeting_agent,
      "News Agent": news_agent
  }
  return agents[agent_name]

In [10]:
# Define the Main AI Agent
main_agent = Agent(
    name = "Main Agent",
    instructions = """
    You are the main assistant.
    Based on the user's request, you decide which specialized agent should handle the task.
    - If the user wants a greeting or a joke, transfer to the Greeting Agent.
    - If the user wants to fetch the news, transfer to the News Agent.
    """,
    functions = [transfer_to_agent]
)

# Start the Connection to the OpenAI API

In [11]:
# Start the connection the OpenAI API
client = Swarm()

In [13]:
# Run the application
response = client.run(
    agent = main_agent,
    messages = [{"role": "user",
                 "content": "Tell me the news in finance"}]
)
print(display(Markdown(response.messages[-1]['content'])))

Here are the top finance news headlines:

1. **Fed Holds Steady on Interest Rates:** The Federal Reserve announced today that it will keep interest rates unchanged, indicating a wait-and-see approach amid economic uncertainties.

2. **Stock Markets Rally:** Global stock markets saw significant gains today, fueled by positive earnings reports from tech giants and optimism over economic recovery.

3. **Inflation Concerns Persist:** Economists warn that inflation rates could remain elevated through the end of the year, urging consumers and businesses to prepare for potential cost increases.

4. **Cryptocurrency Volatility:** Bitcoin and other cryptocurrencies experienced sharp fluctuations in trading, highlighting ongoing volatility in digital assets.

5. **U.S. Job Growth Slows:** Recent data shows a slowdown in job growth in the U.S., raising concerns about the strength of the labor market recovery.

6. **Oil Prices Climb:** Crude oil prices continue to rise, driven by supply constraints and increased global demand.

7. **China's Economic Outlook Weakens:** New reports suggest that China's economic growth may slow down more than anticipated, impacting global trade dynamics.

8. **Corporate Tax Reform Discussions:** Lawmakers are debating potential reforms to corporate tax structures, with implications for multinational companies.

None
